In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import polars as pl
import os
import h5py  # for loading the IDMT data

In [2]:
from sklearn.decomposition import FastICA
import scipy
import scipy.signal as scsig

In [3]:
from synchronization import Synchronizer

## Steps for synchronization
- (0) Load the metadata (Versuchstabelle) -> to identify the file names (soundbook, QASS)
- (1) Load the synchro signals 
    - Dewetron
    - IDMT data, ch3 
- (2) Interpolate the Dewetron signal (to compensate the different sampling rate)
    - (2-0) ICA to increase the SNR
    - (2-1) Linear interpolation
- (3) Cross-correlate the interpolated Dewetron signal and the IDMT data
- (4) Identify the delay (i.e. peak)

## (0) Load the meta data (Versuchstabelle) -> for file names

In [4]:
path_meta = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/00_Versuchsplan_iHub.xlsx'
metadf = pd.read_excel(
    path_meta,
    sheet_name=1,
    usecols="B:F",
    nrows=71
)

## Get the delay for all trials
**info**:<br>
* 27, 28, 29 -> no pre and post in the dewetron!

In [21]:
# Path to the data
path_dew = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/01_DEWETRON_csv'
path_idmt = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/03_Soundbook'
# Base
info_dict = {
    'dataID' : [],
    'DEWETRON': [],
    'SoundBook': [],
    'delay[s]': []
}

for row in np.arange(29, len(metadf)):
    #---- Specify the name
    dataID = row + 1
    fname_dew = metadf['DEWETRON'][row]
    fname_idmt = metadf['SoundBook'][row]
    fname_izfp = metadf['QASS'][row]
    print(f'dataID = {dataID}, name = {fname_dew}')
    
    #---- Synchronize
    synch = Synchronizer()
    # IDMT data
    t_idmt, s_idmt = synch.load_hd5(path_idmt, fname_idmt)
    # Load and process the Dewetron data (denoising + interpoltaion)
    t_dew, s_dew = synch.load_and_process_dew(path_dew, fname_dew, t_idmt, synch.fs_idmt)
    # Identify the delay
    delay, N_offset = synch.identify_delay(s_dew, s_idmt, synch.fs_idmt)
    
    #-------------
    # -> for the trials without pre- and post-triger (27, 28, 29) => use noisy dewetron signal
    # Dewetron data
    # t_dew, s_dew = synch.load_dew(path_dew, fname_dew)
    # # Zero-mean and normalize
    # s_dew = s_dew - np.mean(s_dew)
    # s_dew = s_dew/np.abs(s_dew).max()
    # s_ica = s_dew[:synch.N+1]
    # t_interp, s_interp = synch.interpolate(t_dew, s_ica, t_idmt[int(1.0*synch.fs_idmt)+1:], synch.fs_idmt)
    # # Identify the delay
    # delay, N_offset = synch.identify_delay(s_interp, s_idmt, synch.fs_idmt)
    #-------------
    
    #---- Update the info dictionary
    info_dict['dataID'].append(dataID)
    info_dict['DEWETRON'].append(fname_dew)
    info_dict['SoundBook'].append(fname_idmt)
    info_dict['delay[s]'].append(delay)
    

dataID = 27, name = 27_0_1_Spalt_pneu
dataID = 28, name = 28_0_1_Spalt_pneu
dataID = 29, name = 29_0_1_Spalt_pneu


In [22]:
# Convert the dictionary to a dataframe
info_df = pd.DataFrame.from_dict(info_dict, orient='columns')
# Print
info_df

,dataID,DEWETRON,SoundBook,delay[s]
0,27,27_0_1_Spalt_pneu,iHub2023_030,0.026277
1,28,28_0_1_Spalt_pneu,iHub2023_031,0.026243
2,29,29_0_1_Spalt_pneu,iHub2023_032,0.025291


In [23]:
# Save the dataframe as csv
path2save = '/Volumes/Sandisk_SD/Work/IZFP/Laser/2023_iHub/04_Delays'
info_df.to_csv(f'{path2save}/delays_SoundBook.csv', index=False)
